<a href="https://colab.research.google.com/github/anirxudh/Intelligent-traffic-system-using-fuzzy/blob/main/Traffic_light_automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intelligent Traffic Management System using Fuzzy Logic

### Import libraries and set fuzzy rules

In [17]:
import sys
from typing import List, Tuple, Dict

# Constants for fuzzy sets
WAITING_TRAFFIC = ['minimal', 'light', 'average', 'heavy', 'standstill']
INCOMING_TRAFFIC = ['minimal', 'light', 'average', 'heavy', 'excess']
DURATION = ['short', 'medium', 'long']

# Traffic light timing constants (in seconds)
LIGHT_TIMING = {
    'short': 50,
    'medium': 90,
    'long': 125,
    'default_min': 45,
    'default_max': 150
}

# Fuzzy rules for traffic control
FUZZY_RULES = [
    [['minimal', 'minimal'], 'short'],
    [['minimal', 'light'], 'short'],
    [['minimal', 'average'], 'medium'],
    [['minimal', 'heavy'], 'long'],
    [['minimal', 'excess'], 'long'],
    [['light', 'minimal'], 'short'],
    [['light', 'light'], 'short'],
    [['light', 'average'], 'medium'],
    [['light', 'heavy'], 'medium'],
    [['light', 'excess'], 'long'],
    [['average', 'minimal'], 'short'],
    [['average', 'light'], 'medium'],
    [['average', 'average'], 'medium'],
    [['average', 'heavy'], 'long'],
    [['average', 'excess'], 'long'],
    [['heavy', 'minimal'], 'medium'],
    [['heavy', 'light'], 'medium'],
    [['heavy', 'average'], 'long'],
    [['heavy', 'heavy'], 'long'],
    [['heavy', 'excess'], 'long'],
    [['standstill', 'minimal'], 'medium'],
    [['standstill', 'light'], 'long'],
    [['standstill', 'average'], 'long'],
    [['standstill', 'heavy'], 'long'],
    [['standstill', 'excess'], 'long']
]

### Function: Calculate the fuzzy traffic state based on number of cars.

In [24]:
def calculate_traffic_state(cars: int, is_waiting: bool) -> List[Tuple[str, float]]:
    """
    Args:
        cars: Number of cars
        is_waiting: True if calculating for waiting traffic, False for incoming

    Returns:
        List of tuples containing (traffic_state, membership_value)
    """
    traffic_set = WAITING_TRAFFIC if is_waiting else INCOMING_TRAFFIC

    # Determine which traffic states apply
    states = []
    if 0 <= cars <= 15:
        states.append(traffic_set[0])  # minimal
    if 10 <= cars <= 25:
        states.append(traffic_set[1])  # light
    if 20 <= cars <= 35:
        states.append(traffic_set[2])  # average
    if 30 <= cars <= 45:
        states.append(traffic_set[3])  # heavy
    if cars >= 40:
        states.append(traffic_set[4])  # standstill/excess

    # Calculate membership values
    if len(states) <= 1:
        return [(states[0], 1.0)]

    # Handle overlapping states
    if cars <= 15:
        return [
            (states[0], -(cars - 15) / 5),
            (states[1], -(cars - 25) / 5)
        ]
    elif cars <= 25:
        return [
            (states[0], -(cars - 25) / 5),
            (states[1], (cars - 20) / 5)
        ]
    elif cars <= 35:
        return [
            (states[0], -(cars - 35) / 5),
            (states[1], (cars - 30) / 5)
        ]
    else:
        return [
            (states[0], -(cars - 45) / 5),
            (states[1], (cars - 40) / 5)
        ]

### Fucntion: Apply fuzzy rules to determine traffic light duration.

In [25]:
def apply_fuzzy_rules(waiting: List[Tuple[str, float]],
                     incoming: List[Tuple[str, float]]) -> List[Tuple[str, float]]:

    durations = []
    for w_state, w_value in waiting:
        for i_state, i_value in incoming:
            for rule in FUZZY_RULES:
                if rule[0][0] == w_state and rule[0][1] == i_state:
                    durations.append((rule[1], min(w_value, i_value)))
    return durations

### Function: Convert fuzzy duration states to actual seconds

In [26]:
def calculate_duration(duration_states: List[Tuple[str, float]]) -> int:

    if not duration_states:
        return LIGHT_TIMING['default_min']

    # Calculate weighted average of durations
    total_weight = sum(weight for _, weight in duration_states)
    if total_weight == 0:
        return LIGHT_TIMING['default_min']

    weighted_sum = sum(
        LIGHT_TIMING[state] * weight
        for state, weight in duration_states
    )

    duration = weighted_sum / total_weight

    # Ensure duration stays within reasonable bounds
    return min(max(int(duration), LIGHT_TIMING['default_min']),
              LIGHT_TIMING['default_max'])

### Function: Process traffic situation and determine green light duration.

In [21]:
def process_traffic(waiting_cars: int, incoming_cars: int) -> int:
    """
    Args:
        waiting_cars: Number of cars waiting at the light
        incoming_cars: Number of cars approaching intersection

    Returns:
        Recommended green light duration in seconds
    """
    # Get fuzzy states for both waiting and incoming traffic
    waiting_state = calculate_traffic_state(waiting_cars, True)
    incoming_state = calculate_traffic_state(incoming_cars, False)

    # Apply fuzzy rules and calculate duration
    duration_states = apply_fuzzy_rules(waiting_state, incoming_state)
    return calculate_duration(duration_states)

In [22]:
def main():
    """Main function to demonstrate the traffic light system."""
    # Example traffic scenario
    waiting_cars = 10
    incoming_cars = 33

    print("\n=== Intelligent Traffic Light System ===\n")
    print(f"Current Traffic Situation:")
    print(f"- Cars waiting at light: {waiting_cars}")
    print(f"- Cars approaching intersection: {incoming_cars}\n")

    # Calculate waiting and incoming traffic states
    waiting_state = calculate_traffic_state(waiting_cars, True)
    incoming_state = calculate_traffic_state(incoming_cars, False)

    print(f"Analysis Results:")
    print(f"- Waiting traffic state: {waiting_state}")
    print(f"- Incoming traffic state: {incoming_state}")

    # Calculate final duration
    duration = process_traffic(waiting_cars, incoming_cars)

    print(f"\nRecommended Action:")
    print(f"→ Set traffic light to GREEN for {duration} seconds")

In [23]:
if __name__ == '__main__':
    main()


=== Intelligent Traffic Light System ===

Current Traffic Situation:
- Cars waiting at light: 10
- Cars approaching intersection: 33

Analysis Results:
- Waiting traffic state: [('minimal', 1.0), ('light', 3.0)]
- Incoming traffic state: [('average', 0.4), ('heavy', 0.6)]

Recommended Action:
→ Set traffic light to GREEN for 100 seconds
